In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch
import numpy as np
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
import pytorch_lightning as pl

/home/fujiwara/miniconda3/envs/compe_diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
from transformers import BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [4]:
def load_label(trg):
    label = []
    with open("data/{}/{}.label".format(trg, trg), "r") as f:
        lines = f.read().rstrip().split("\n")
        for line in lines:
            d1, d2 = map(int, line.split("\t"))
            label.append(d1 - 1)
    return np.array(label, dtype=int)

train_label = load_label("train")
dev_label = load_label("dev")
test_label = np.zeros(1843, dtype=int)

print(len(train_label), len(dev_label), len(test_label))

13990 1843 1843


In [5]:
print(train_label.min(), dev_label.min(), test_label.max())
print(train_label.max(), dev_label.max(), test_label.max())

0 0 0
5 5 0


In [6]:
def load_text(trg):
    with open("data/{}/{}.txt".format(trg, trg), "r") as f:
        lines = f.read().rstrip().split("\n")
    return np.array(lines, dtype=object)

train_txt = load_text("train")
dev_txt = load_text("dev")
test_txt = load_text("test")

print(len(train_txt), len(dev_txt), len(test_txt))

13990 1843 1843


In [7]:
print(train_txt[0])

Concussions prevented him from playing longer .


In [8]:
def get_dataset(txts, labels):
    dataset = []
    for txt, label in zip(txts, labels):
        mydict = tokenizer(txt, max_length=32, padding="max_length", truncation=True)
        mydict["labels"] = label
        mydict = {key: torch.tensor(value) for key, value in mydict.items()}
        dataset.append(mydict)
    return dataset

train_dataset = get_dataset(train_txt, train_label)
dev_dataset = get_dataset(dev_txt, dev_label)
test_dataset = get_dataset(test_txt, test_label)

In [9]:
print(train_dataset[0])

{'input_ids': tensor([  101, 23159,  2015,  8729,  2032,  2013,  2652,  2936,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), 'labels': tensor(1)}


In [10]:
dataloader_train = DataLoader(train_dataset, batch_size=32, shuffle=True)
dataloader_val = DataLoader(dev_dataset, batch_size=256, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [11]:
# ====================
# BERTによるテキスト分類
# ====================

class Bert4Classification(pl.LightningModule):

    # モデルの読み込みなど。損失関数は自動的に設定される。
    # num_labels == 1 -> 回帰タスクなので MSELoss()
    # num_labels > 1 -> 分類タスクなので CrossEntropyLoss()
    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()    # num_labelsとlrを保存する。例えば、self.hparams.lrでlrにアクセスできる。
        self.bert_sc = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    # 訓練用データのバッチを受け取って損失を計算
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log("train_loss", loss)
        return loss

    # 検証用データのバッチを受け取って損失を計算
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log("val_loss", val_loss)

    # 評価用データのバッチを受け取って分類の正解率を計算
    def test_step(self, batch, batch_idx):
        # ラベルの推定
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        # 正解率の計算
        labels = batch.pop("labels")
        num_correct = (labels_predicted == labels).sum().item()
        accuracy = num_correct / labels.size(0)
        self.log("accuracy", accuracy)

    # 最適化手法を設定
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [12]:
# ====================
# 訓練
# ====================
model_name = 'bert-base-uncased'
model = Bert4Classification(model_name, num_labels=6, lr=0.0001)

# 訓練中にモデルを保存するための設定
checkpoint = pl.callbacks.ModelCheckpoint(
    # 検証用データにおける損失が最も小さいモデルを保存する
    monitor="val_loss", mode="min", save_top_k=1,
    # モデルファイル（重みのみ）を "model" というディレクトリに保存する
    save_weights_only=True, dirpath="model/"
)

early_stopping = pl.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3)

# 訓練
trainer = pl.Trainer(accelerator="auto", max_epochs=20, callbacks=[checkpoint, early_stopping])
trainer.fit(model, dataloader_train, dataloader_val)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/home/fujiwara/miniconda3/envs/compe_diff/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/fujiwara/miniconda3/envs/compe_diff/lib/python ...
GPU available: True (cuda), used: True
TPU available: False, using: 

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/fujiwara/miniconda3/envs/compe_diff/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/fujiwara/miniconda3/envs/compe_diff/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 4: 100%|██████████| 438/438 [00:41<00:00, 10.51it/s, v_num=0]
ベストモデル:  /net/nas8/data/home/fujiwara/competition/difficulty/model/epoch=1-step=876.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.8834, device='cuda:0')


In [13]:
with torch.no_grad():
    preds = list()
    for batch in dataloader_test:
        output = model.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        preds.append(labels_predicted)
    preds = torch.cat(preds)
    pred_label = [label.item() for label in preds]
preds

tensor([2, 1, 3,  ..., 3, 3, 3])

In [14]:
with open("out.txt", "w") as f:
    preds = preds.tolist()
    for pred in preds:
        f.write("{}\n".format(pred+1))
